# Artificial Neural Network

### Importing the libraries

In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

In [ ]:
print(y)

### Encoding categorical data

Label Encoding the "Gender" column

In [ ]:
# No missing data in dataset

# Since gender is binary value
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    transformers = [(
        'encoder',
        OneHotEncoder(),
        [1]                 # index of column to apply one hot encoding
    )],
    remainder = 'passthrough'
)

X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
# Feature scaling is fundamental for deep learning even for dummy variables
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()              # Initialsing the artifical Neural network using Sequential Class

### Adding the input layer and the first hidden layer

In [ ]:
# Adding a layer in network
ann.add(tf.keras.layers.Dense(
    units = 6,                                   # Number of neurons which is based on experiments
    activation = 'relu'                          # Rectifier activation function 
))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(
    units = 6,                                   
    activation = 'relu'                          
))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(
    units = 1,                                   
    activation = 'sigmoid'                          
))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# categorical_crossentropy for categorical outcome

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 50)

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of single observation

In [ ]:
singleTest = np.array([[
    600,
    'France',
    'Male',
    40,
    3,
    60000,
    2,
    1,
    1,
    50000
]])
# print(singleTest)

singleTest[:, 2] = le.transform(singleTest[:, 2])
singleTest = np.array(ct.transform(singleTest))

singleTest = sc.transform(singleTest)

In [ ]:
print(singleTest)

In [ ]:
ann.predict(singleTest)

In [ ]:
# Mentor solution 
ann.predict(sc.transform([[
    1, 0, 0,
    600,
    1,
    40,
    3,
    60000,
    2,
    1,
    1,
    50000
]]))

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)